## Lab 6 - MlInspect

In [1]:
#install python 3.8 and dev utils
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-dev python3.8-distutils libpython3.8-dev

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2

!python3 --version

!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

!ln -s /usr/local/lib/python3.9/dist-packages/google \
       /usr/local/lib/python3.8/dist-packages/google

!sed -i "s/from IPython.utils import traitlets as _traitlets/import traitlets as _traitlets/" /usr/local/lib/python3.8/dist-packages/google/colab/*.py
!sed -i "s/from IPython.utils import traitlets/import traitlets/" /usr/local/lib/python3.8/dist-packages/google/colab/*.py

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.8-distutils'
libpython3.8-dev is already the ne

In [2]:
# check python version -- should be 3.8
import sys
print(sys.version)
!python3 --version
!python --version

3.8.10 (default, Mar 13 2023, 10:26:41) 
[GCC 9.4.0]
Python 3.8.10
Python 3.8.10


## Install package required.

In [3]:
# graphviz is installed already, but need lib too
!apt install libgraphviz-dev
!pip install pygraphviz
!pip install mlinspect
!pip install tensorflow==2.5.0
!pip install keras==2.4.3
!pip install jinja2==3.0.3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgraphviz-dev is already the newest version (2.42.2-3build2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ipython-7.19.0-py3-none-any.whl (784 kB)
  Attempting uninstall: ipython
    Found existing installation: ipython 8.11.0
    Uninstalling ipython-8.11.0:
      Successfully uninstalled ipython-8.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipykernel 6.22.0 requires ipython>=7.23.1, but you have ipython 7.19.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [ ]:
#@title This cell will restart the environment so you don't have to do it manually. Continue from here after restart.
import os
os.kill(os.getpid(), 9)

## Overview of the example from the paper


![paper_example_image](https://github.com/stefan-grafberger/mlinspect/blob/master/demo/feature_overview/paper_example_image.png?raw=true)

Example of an ML pipeline that predicts which patients are at a higher risk of serious complications, under the requirement to achieve comparable false negative rates across intersectional groups by age and race. The pipeline is implemented using native constructs from the popular pandas and scikit-learn libraries. On the left, we highlight potential issues identified by mlinspect. On the right, we show the corresponding dataflow graph extracted by mlinspect to instrument the code and pinpoint issues.

## Task
Operators like joins, selections and missing value imputaters can cause data distribution issues, which can heavily impact the performance of our model for specific demographic groups. Mlinspect helps with identifying such issues by offering a check that calculates histograms for sensitive groups in the data and verifying whether the histogram change is significant enough to alert the user. Thanks to our annotation propagation, we can deal with complex code involving things like nested sklearn pipelines and group memberships that are removed from the training data using projections.

We want to find out if preprocessing operations in pipelines introduce bias and if so, which groups are effected. The pipeline we want to analyse in this task can be found using the path os.path.join(str(get_project_root()), "experiments", "user_interviews", "adult_simple_modified.py"). The senstive attributes we want to take a look at are race and sex.

It is using a benchmark dataset frequently used in the algorithmic fairness literature. Adult income contains information about 33,000 individuals from the 1994 U.S. census, with sensitive attributes gender and race. The corresponding task is to predict whether the annual income of an individual exceeds $50,000. We took this existing data set and only modified it slightly by introducing an artificial issue which we will now try to find using mlinspect.

The code of the pipeline:

> ```python
> """
> Adult income pipeline
> """
> import os
> import pandas as pd
> from sklearn import compose, preprocessing, tree, pipeline
> 
> from mlinspect.utils import get_project_root
> 
> print('pipeline start')
> 
> train_file_a = os.path.join(str(get_project_root()), "experiments", "user_interviews", "adult_simple_train_a.csv")
> raw_data_a = pd.read_csv(train_file_a, na_values='?', index_col=0)
> 
> train_file_b = os.path.join(str(get_project_root()), "experiments", "user_interviews", "adult_simple_train_b.csv")
> raw_data_b = pd.read_csv(train_file_b, na_values='?', index_col=0)
> 
> merged_raw_data = raw_data_a.merge(raw_data_b, on="id")
> 
> data = merged_raw_data.dropna()
> 
> labels = preprocessing.label_binarize(data['income-per-year'], classes=['>50K', '<=50K'])
> 
> column_transformer = compose.ColumnTransformer(transformers=[
>     ('categorical', preprocessing.OneHotEncoder(handle_unknown='ignore'), ['education', 'workclass']),
>     ('numeric', preprocessing.StandardScaler(), ['age', 'hours-per-week'])
> ])
> adult_income_pipeline = pipeline.Pipeline([
>     ('features', column_transformer),
>     ('classifier', tree.DecisionTreeClassifier())])
> 
> adult_income_pipeline.fit(data, labels)
> print('pipeline finished')
> ```

Additional reading:
https://scikit-learn.org/stable/modules/compose.html#combining-estimators

# Step 1/4: Add check and execute the pipeline

The central entry point of mlinspect is the `PipelineInspector`. To use mlinspect, we use it and pass it the path to the runnable version of the example pipeline. Here, we have the example pipeline in a `.py` file. 

First, we define the check we want mlinspect to run. In this example, we only use `NoBiasIntroducedFor(["col1", "col2", ...])` to automatically check for significant changes in the distribution of sensitive demograhpic groups and compute the histograms.

Then, we execute the pipeline. Mlinspect returns a `InspectorResult`, which, among other information, contains the output of our check. 

In [1]:
import urllib
from inspect import cleandoc

import os
from mlinspect.utils import get_project_root

from mlinspect import PipelineInspector
from mlinspect.checks import NoBiasIntroducedFor, NoIllegalFeatures
from mlinspect.inspections import MaterializeFirstOutputRows

2023-03-23 15:34:55.844833: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
python_string = cleandoc("""
import os
import pandas as pd
from sklearn import compose, preprocessing, tree, pipeline

from mlinspect.utils import get_project_root

print('pipeline start')
raw_data_a = pd.read_csv('https://raw.githubusercontent.com/DataResponsibly/Datasets/master/dataset_lab_6/adult_simple_train_a.csv')
raw_data_b = pd.read_csv('https://raw.githubusercontent.com/DataResponsibly/Datasets/master/dataset_lab_6/adult_simple_train_b.csv')

merged_raw_data = raw_data_a.merge(raw_data_b, on="id")

data = merged_raw_data.dropna()

labels = preprocessing.label_binarize(data['income-per-year'], classes=['>50K', '<=50K'])

column_transformer = compose.ColumnTransformer(transformers=[
    ('categorical', preprocessing.OneHotEncoder(handle_unknown='ignore'), ['education', 'workclass']),
    ('numeric', preprocessing.StandardScaler(), ['age', 'hours-per-week'])
])
adult_income_pipeline = pipeline.Pipeline([
    ('features', column_transformer),
    ('classifier', tree.DecisionTreeClassifier())])

adult_income_pipeline.fit(data, labels)
print('pipeline finished')
""")

In [3]:
inspector_result = PipelineInspector\
    .on_pipeline_from_string(python_string) \
    .add_check(NoBiasIntroducedFor(['race', 'sex'])) \
    .add_check(NoIllegalFeatures()) \
    .add_required_inspection(MaterializeFirstOutputRows(5)) \
    .execute()

extracted_dag = inspector_result.dag
check_results = inspector_result.check_to_check_results
inspection_results = inspector_result.inspection_to_annotations

pipeline start
pipeline finished


In [4]:
extracted_dag.nodes

NodeView((DagNode(node_id=7, operator_type=<OperatorType.DATA_SOURCE: 'Data Source'>, code_reference=CodeReference(lineno=8, col_offset=13, end_lineno=8, end_col_offset=132), module=('pandas.io.parsers', 'read_csv'), description='adult_simple_train_a.csv', columns=['Unnamed: 0', 'id', 'age', 'workclass', 'fnlwgt', 'education', 'education-num']), DagNode(node_id=10, operator_type=<OperatorType.DATA_SOURCE: 'Data Source'>, code_reference=CodeReference(lineno=9, col_offset=13, end_lineno=9, end_col_offset=132), module=('pandas.io.parsers', 'read_csv'), description='adult_simple_train_b.csv', columns=['Unnamed: 0', 'id', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']), DagNode(node_id=14, operator_type=<OperatorType.JOIN: 'Join'>, code_reference=CodeReference(lineno=11, col_offset=18, end_lineno=11, end_col_offset=55), module=('pandas.core.frame', 'merge'), description='on id', columns=['

# Step 2/4: Overview of the check results
## Did our check find issues?

Let us look at the `check_result` to see our some check failed. We do this using the mlinspect utlity function `check_results_as_data_frame(...)`. We see that an issue was found, so we have to investigate it.

In [5]:
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

check_result_df = PipelineInspector.check_results_as_data_frame(check_results)
display(check_result_df)

,check_name,status,description
0,"NoBiasIntroducedFor((('race', 'sex'), -0.3))",CheckStatus.FAILURE,"A Join causes a min_relative_ratio_change of 'sex' by -0.4009821195113874, a value below the configured minimum threshold -0.3!"
1,NoIllegalFeatures(),CheckStatus.FAILURE,Used illegal columns: ['age']


A negative min_relative_ratio_change means that the ratio after the join is less than the original ratio

https://github.com/stefan-grafberger/mlinspect/blob/19ca0d6ae8672249891835190c9e2d9d3c14f28f/mlinspect/checks/_no_bias_introduced_for.py

# Step 3/4: List of operations that could change the distribution 

As stated before, only some operations like selections, joins and missing value imputation can change the distribution. Our check already filtered all operators that can cause data distribution issues. We can use the mlinspect utility function `get_distribution_changes_overview_as_df(...)` to get an overview. The overview already tells us that mlinspect detected a potential issue caused by a JOIN involving the gender attribute. Note that the automatic issue detection from mlinspect is only as good as its configuration and should not be completely relied upon.

In [6]:
no_bias_check_result = check_results[NoBiasIntroducedFor(["race", "sex"])]

distribution_changes_overview_df = NoBiasIntroducedFor.get_distribution_changes_overview_as_df(no_bias_check_result)
display(distribution_changes_overview_df)

dag_node_distribution_changes_list = list(no_bias_check_result.bias_distribution_change.items())

,DagNode OperatorType,DagNode CodeReference,DagNode Module,DagNode Description,'race' distribution change below the configured minimum test threshold,'sex' distribution change below the configured minimum test threshold
0,OperatorType.JOIN,"CodeReference(lineno=11, col_offset=18, end_lineno=11, end_col_offset=55)","(pandas.core.frame, merge)",on id,False,True
1,OperatorType.SELECTION,"CodeReference(lineno=13, col_offset=7, end_lineno=13, end_col_offset=31)","(pandas.core.frame, dropna)",dropna,False,False


# Step 4/4 Detailed Investigation

### NoBiasIntroduced
Now that we know of the potential issue, we will take a look at the histograms before and after the JOIN. We can use `distribution_change.before_and_after_df` to look at the data in form of a `pandas.DataFrame` or use the mlinspect utility function `plot_distribution_change_histograms(...)` to plot the histograms. 

In [ ]:
# Select the DagNode we want to look at by index
dag_node, node_distribution_changes = dag_node_distribution_changes_list[0]

# Investige the changes
print(dag_node.operator_type)
for column, distribution_change in node_distribution_changes.items():
    print("")
    print("\033[1m Column '{}'\033[0m, acceptable change: {}, min_relative_ratio_change: {}".format(column, distribution_change.acceptable_change, distribution_change.min_relative_ratio_change))
    display(distribution_change.before_and_after_df)
    NoBiasIntroducedFor.plot_distribution_change_histograms(distribution_change)

### Use of illegal features: 

https://github.com/stefan-grafberger/mlinspect/blob/19ca0d6ae8672249891835190c9e2d9d3c14f28f/mlinspect/checks/_no_illegal_features.py

In [ ]:
feature_check_result = check_results[NoIllegalFeatures()]
print("Used illegal features: {}".format(feature_check_result.illegal_features))

### MaterializeFirstOutputRows
For each operator, the MaterializeFirstOutputRows materialized the first 5 output rows. Especially for scikit-learn pipelines, it requires custom debugging code if a user just wants to look at some intermediate results ([see example stackoverflow post](https://stackoverflow.com/questions/34802465/sklearn-is-there-any-way-to-debug-pipelines)). Using mlinspect, this becomes easy. We can look at the input and output of arbitrary featurizers like OneHotEncoders or Word2Vec models.

Here, we use this functionality to look at the output of a OneHotEncoder and the imputer right before it. For this, we only need to look at the inspection result for the corresponding Dag nodes.

In [ ]:
from IPython.display import display

first_rows_inspection_result = inspection_results[MaterializeFirstOutputRows(5)]

relevant_nodes = [node for node in extracted_dag.nodes if node.description in {
    "Categorical Encoder (OneHotEncoder), Column: 'education'", "Categorical Encoder (OneHotEncoder), Column: 'workclass'" }]

for dag_node in relevant_nodes:
    if dag_node in first_rows_inspection_result and first_rows_inspection_result[dag_node] is not None:
        print("\n\033[1m{} ({})\033[0m\n{}".format(
            dag_node.operator_type, dag_node.description, dag_node.code_reference))
        display(first_rows_inspection_result[dag_node])

# [TODO] Questions: 

* *Did we find operators that introduce bias? How 
did the distribution of demographic groups change?*



**Write down your answer here:** An error message "NoBiasIntroducedFor((('race', 'sex'), -0.3))" suggests that an analysis or model introduced bias with respect to "race" and "sex". It is likely that the distribution of demographic groups was affected, resulting in unequal outcomes for different groups. To fix this error, adjustments to the analysis or model may be needed, such as reviewing the data, adjusting the model parameters, or incorporating additional variables. It may also be necessary to consult with experts or seek guidance from the software documentation or support team to ensure that the analysis is conducted in a fair and ethical manner.

* *Can missing value imputation cause bias? If so, 
how?*


**Write down your answer here:**Missing value imputation can introduce bias if not performed appropriately, as missing data is often not missing at random. Imputing missing values based on the observed data can result in over or under-representation of certain groups in the final analysis, leading to biased estimates. Appropriate methods should be used to ensure fair and ethical analysis.